# AuctiML - оценка торгов



In [224]:
import pandas as pd
import numpy as np
import json

In [486]:
# pd.options.display.max_cols = 999
pd.options.display.max_columns = None

### Загрузка данных

In [469]:
PATH = "../"
PATH_DATASET = PATH + "datasets/"
torgi_filename = PATH_DATASET + "torgi_dataset.csv"
torgi_output_filename = PATH_DATASET + "preproc_torgi_dataset.csv"

In [460]:
torgi_df = pd.read_csv(torgi_filename, index_col='id', encoding='utf8')
torgi_df.shape

(9996, 56)

### Предобработка данных

In [461]:
def str2json(string):
    string = string.replace("\\xa0", ' ').replace("\\x20", '" "')
    string = string.replace("\"", "").replace("\'", "\"").replace("False", "\"False\"").replace("True", "\"True\"")
    return json.loads(string)

In [487]:
torgi_df

,lotName,lotStatus,biddForm,noticeNumber,lotNumber,biddType,subjectRFCode,lotDescription,lotImages,lotAttachments,noticeAttachments,currencyCode,category,timeZoneName,timezoneOffset,ownershipForm,estateAddress,hasAppeals,isStopped,biddStartTime,biddEndTime,versionId,noticeSignedData,egrkns,egrnInfoRequestList,noticeFirstVersionPublicationDate,isAnnulled,priceMin,priceStep,prevBiddInfo,previousProcedures,etpCode,etpUrl,deposit,depositElectronicPlatform,depositRecipientName,depositRecipientINN,depositRecipientKPP,depositBankName,depositBIK,depositPayAccount,depositCorAccount,depositPurposePayment,auctionStartDate,depositTreasuryAccount,isCompound,noticeAttributes,privatizationReason,cancellation,lotToPlanLink,protocols,cutoffPrice,priceDecreaseStep,suspensions,Вид разрешённого использования земельного участка,Границы лесного участка,Площадь земельного участка,Площадь земельного участка в соответствии,Регистрационный номер ЕГРОКН,Условный номер земельного участка,Назначение земельного участка,Кадастровый номер земельного участка,Обременения прав на земельный участок,Максимально допустимые параметры разрешенного строительства,Вид договора,Адрес и время приема граждан,Информация об обременениях лесных участков,Информация об ограничении использования лесов,Для сельскохозяйственных организаций,Ежегодный платеж за объект,Платеж за право заключить договор,Собственник имущества,Реквизиты решения о проведении торгов,Вид предмета торгов,Права на земельный участок,Ежемесячный платеж за объект,"Право лица, с которым заключен",Порядок и сроки уплаты покупной,Возможна субаренда,Информация об объеме подлежащей заготовке,Информация о состоянии земельного участка,"Условия договора, заключаемого по результатам",Реквизиты решения об утверждении проекта,Цель предоставления земельного участка,Минимальный объем осуществления жилищного строительства,Льгота по арендной плате,Информация о сроках и порядке,Дата принятия решения о проведении,Требования к параметрам и характеристикам,Порядок возврата задатка,Для граждан,Возможность подключения (технологического присоединения) объектов,Основание реализации имущества,"Обременения, ограничения",Сведения о собственнике (правообладателе) земельного,Порядок оплаты,"Порядок ознакомления с имуществом, иной",Порядок ознакомления с имуществом,Срок и порядок внесения задатка,Основание реализации имущества,Для субъектов малого и среднего,Сведения о решении о проведении,Срок действия договора - месяцев,Обременения реализуемого имущества,Только для крестьянских (фермерских) хозяйств,Срок заключения договора,Только для МСП,"Информация об официальном сайте, на",Предмет торга,Срок аренды,Начальная цена за кв.м,Для крестьянских (фермерских) хозяйств,Минимально допустимые параметры разрешенного строительства,Информация об обязательствах по приведению,Информация о порядке внесения участниками,Обременения продаваемого имущества,Срок действия договора - дней,Адрес сайта в информационно-телекоммуникационной сети,Целевое назначение государственного или муниципального,Сдача имущества в субаренду,Размер взимаемой с победителя аукциона,Информация о выявленных в отношении,Ограничения в использовании земельного участка,Срок действия договора - лет,Участниками аукциона могут являться только,Информация о сроке аренды лесного,Информация об обязательствах по сносу,Перечисление задатка,Особенности объекта торгов,Только для граждан,Ограничения прав на земельный участок
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21000000160000000069_1,Прием заявлений о намерении участвовать в аукц...,SUBMISSION,Сообщение о предоставлении (реализации),21000000160000000069,1,Аренда и продажа земельных участков,52,Прием заявлений о намерении участвовать в аукц...,['64edf46105f9fa758111de88'],"[{'fileId': '64edf46105f9fa758111de88', 'signa...","[{'fileId': '64edf45355ca906d57ab7378', 'signa...",643,Земли сельскохозяйственного назначения,МСК,180,Иная собственность,"обл Ниже

In [463]:
# Переименовываем значения в более удобные
torgi_df['lotStatus'] = torgi_df['lotStatus'].replace({'APPLICATIONS_SUBMISSION':'SUBMISSION', 'DETERMINING_WINNER':'WINNER'})
torgi_df['lotStatus'].unique()

array(['SUBMISSION', 'PUBLISHED', 'CANCELED', 'SUCCEED', 'WINNER',
       'FAILED'], dtype=object)

In [464]:
# Форма и тип торгов и другие атрибуты предствалены в виде json с указаием кода и наименования торгов, выделим наименование 
torgi_df['biddForm'] = torgi_df['biddForm'].apply(lambda x: str2json(x)['name'])
torgi_df['biddType'] = torgi_df['biddType'].apply(lambda x: str2json(x)['name'])
torgi_df['category'] = torgi_df['category'].apply(lambda x: str2json(x)['name'])
torgi_df['ownershipForm'] = torgi_df['ownershipForm'].apply(lambda x: str2json(x)['name'])
torgi_df['egrkns'] = torgi_df['egrkns'].apply(lambda x: '' if x is np.nan else str2json(x)[0]['number'])
torgi_df['egrnInfoRequestList'] = torgi_df['egrnInfoRequestList'].apply(lambda x: '' if x is np.nan or len(str2json(x)) == 0 else str2json(x)[0]['objects'][0]['cadastralNumber'])

In [465]:
charact_columns = ['Вид разрешённого использования земельного участка', 
           'Границы лесного участка', 'Площадь земельного участка', 
           'Площадь земельного участка в соответствии с проектом межевания территории или со схемой расположения ', 
           'Регистрационный номер ЕГРОКН', 
           'Условный номер земельного участка', 
           'Назначение земельного участка', 
           'Кадастровый номер земельного участка']
def get_characters(x:list, charact_columns:list):
    item_attribute = {}
    for i in x:
        if i['name'] in ['Вид разрешённого использования земельного участка']:
            item_value = [val['value'] for val in i['characteristicValue']]
            item_attribute[i['name']] = item_value
        elif i['name'] in ['Назначение земельного участка']:
            item_attribute[i['name']] = i['characteristicValue']["value"]
        elif i['name'] in ['Кадастровый номер земельного участка','Границы лесного участка', 'Условный номер земельного участка', 'Площадь земельного участка', 'Регистрационный номер ЕГРОКН', 'Площадь земельного участка в соответствии с проектом межевания территории или со схемой расположения ']:
            if 'characteristicValue' in i:
                if isinstance(i['characteristicValue'], list):
                    print('Кадастровый номер is list')
                    assert False
                if i['name'] in ['Площадь земельного участка', 'Границы лесного участка''Площадь земельного участка в соответствии с проектом межевания территории или со схемой расположения '] and i['unit']['name'] != 'Квадратный метр':
                    raise ValueError(f"Площадь земельного участка != Квадратный метр ({i['unit']['name']})")
                item_attribute[i['name']] = i['characteristicValue']
    result_values = []
    for col in charact_columns:
        val = item_attribute.get(col, np.nan)
        result_values.append(val)
    return result_values

named_charact_columns = [" ".join(key.split(" ")[:5]) for key in charact_columns]
torgi_df[named_charact_columns] = torgi_df['characteristics'].apply(lambda x: pd.Series(get_characters(str2json(x), charact_columns)))
torgi_df.drop(columns=['characteristics'])
torgi_df[named_charact_columns].isna().sum()

Вид разрешённого использования земельного участка       0
Границы лесного участка                              9546
Площадь земельного участка                            651
Площадь земельного участка в соответствии            9345
Регистрационный номер ЕГРОКН                         9390
Условный номер земельного участка                    9649
Назначение земельного участка                        9345
Кадастровый номер земельного участка                  651
dtype: int64

In [466]:
attrib_columns = ['Обременения прав на земельный участок', 'Максимально допустимые параметры разрешенного строительства объекта капитального строительства (при необходимости)', 
                  'Вид договора', 'Адрес и время приема граждан для ознакомления со схемой расположения земельного участка, в соответствии с которой предстоит образовать земельный участок, если данная схема представлена на бумажном носителе', 
                  'Информация об обременениях лесных участков', 'Информация об ограничении использования лесов', 'Для сельскохозяйственных организаций', 'Ежегодный платеж за объект', 'Платеж за право заключить договор', 'Собственник имущества', 
                  'Реквизиты решения о проведении торгов', 'Вид предмета торгов', 'Права на земельный участок', 'Ежемесячный платеж за объект', 'Право лица, с которым заключен договор аренды земельного участка единого института развития, выкупить такой земельный участок', 
                  'Порядок и сроки уплаты покупной цены по итогам проведения торгов', 'Возможна субаренда', 'Информация об объеме подлежащей заготовке древесины.', 'Информация о состоянии земельного участка', 'Условия договора, заключаемого по результатам торгов', 
                  'Реквизиты решения об утверждении проекта межевания территории', 'Цель предоставления земельного участка', 'Минимальный объем осуществления жилищного строительства', 'Льгота по арендной плате', 'Информация о сроках и порядке внесения задатка, реквизитах счета для перечисления задатка указана в документации (при наличии требования о внесении задатка)', 
                  'Дата принятия решения о проведении аукциона', 'Требования к параметрам и характеристикам планируемого развития территории', 'Порядок возврата задатка', 'Для граждан', 'Возможность подключения (технологического присоединения) объектов капитального строительства к сетям инженерно-технического обеспечения (за исключением сетей электроснабжения) ', 
                  'Основание реализации имущества', 'Обременения, ограничения', 'Сведения о собственнике (правообладателе) земельного участка', 'Информация об объеме подлежащей заготовке древесины', 'Порядок оплаты', 'Порядок ознакомления с имуществом, иной информацией', 'Порядок ознакомления с имуществом', 'Срок и порядок внесения задатка', 
                  'Основание реализации имущества ', 'Для субъектов малого и среднего предпринимательства', 'Сведения о решении о проведении аукциона', 'Срок действия договора - месяцев ', 'Обременения реализуемого имущества', 'Только для крестьянских (фермерских) хозяйств', 'Срок заключения договора', 'Только для МСП', 
                  'Информация об официальном сайте, на котором размещена документация об аукционе', 'Предмет торга', 'Срок аренды', 'Начальная цена за кв.м', 'Для крестьянских (фермерских) хозяйств', 'Минимально допустимые параметры разрешенного строительства объекта капитального строительства (при необходимости)', 
                  'Информация об обязательствах по приведению в соответствие с установленными требованиями здания, сооружения, объекта незавершенного строительства, которые расположены на земельном участке', 'Информация о порядке внесения участниками аукциона и возврата им задатка', 
                  'Обременения продаваемого имущества', 'Срок действия договора - дней', 'Адрес сайта в информационно-телекоммуникационной сети Интернет, на котором размещен утвержденный проект межевания территории', 'Целевое назначение государственного или муниципального имущества, права на которое передаются по договору', 
                  'Сдача имущества в субаренду', 'Размер взимаемой с победителя аукциона или иных лиц, с которыми заключается договор, платы оператору электронной площадки (размер устанавливается в соответствии с постановлением Правительства РФ от 10.05.2018 № 564)', 
                  'Информация о выявленных в отношении земельного участка в рамках федерального государственного земельного контроля (надзора) нарушениях обязательных требований', 'Ограничения в использовании земельного участка', 'Срок действия договора - лет', 
                  'Участниками аукциона могут являться только субъекты малого и среднего предпринимательства, физические лица, применяющие специальный налоговый режим Налог на профессиональный доход, имеющие право на поддержку органов государственной власти и органов местного самоуправления', 
                  'Информация о сроке аренды лесного участка', 'Информация об обязательствах по сносу здания, сооружения, объекта незавершенного строительства, которые расположены на земельном участке', 'Перечисление задатка', 'Особенности объекта торгов', 'Только для граждан', 
                  'Ограничения прав на земельный участок', 'Срок действия договора - месяцев'
                 ]

def get_attributes(x:list, attrib_columns:list):
    item_attribute = {}
    for i in x:
        if i['fullName'] in attrib_columns:
            if 'value' in i:
                if isinstance(i['value'], dict):
                    item_attribute[i['fullName']] = i['value']['name']
                else:
                    item_attribute[i['fullName']] = i['value']
    result_values = []
    for col in attrib_columns:
        val = item_attribute.get(col, np.nan)
        result_values.append(val)
    return result_values

named_attrib_columns = [" ".join(key.split(" ")[:5]) for key in attrib_columns]
torgi_df[named_attrib_columns] = torgi_df['attributes'].apply(lambda x: pd.Series(get_attributes(str2json(x), attrib_columns)))
torgi_df.drop(columns=['attributes'])
torgi_df[named_attrib_columns].isna().sum()

Обременения прав на земельный участок                                9992
Максимально допустимые параметры разрешенного строительства          7205
Вид договора                                                         5098
Адрес и время приема граждан                                         7294
Информация об обременениях лесных участков                           9514
Информация об ограничении использования лесов                        9407
Для сельскохозяйственных организаций                                 9822
Ежегодный платеж за объект                                           9982
Платеж за право заключить договор                                    9973
Собственник имущества                                                9504
Реквизиты решения о проведении торгов                                9336
Вид предмета торгов                                                  9337
Права на земельный участок                                           7667
Ежемесячный платеж за объект          

In [476]:
torgi_df = torgi_df.drop(columns=['attributes', 'characteristics'])

In [477]:
torgi_df

,lotName,lotStatus,biddForm,noticeNumber,lotNumber,biddType,subjectRFCode,lotDescription,lotImages,lotAttachments,noticeAttachments,currencyCode,category,timeZoneName,timezoneOffset,ownershipForm,estateAddress,hasAppeals,isStopped,biddStartTime,biddEndTime,versionId,noticeSignedData,egrkns,egrnInfoRequestList,noticeFirstVersionPublicationDate,isAnnulled,priceMin,priceStep,prevBiddInfo,previousProcedures,etpCode,etpUrl,deposit,depositElectronicPlatform,depositRecipientName,depositRecipientINN,depositRecipientKPP,depositBankName,depositBIK,depositPayAccount,depositCorAccount,depositPurposePayment,auctionStartDate,depositTreasuryAccount,isCompound,noticeAttributes,privatizationReason,cancellation,lotToPlanLink,protocols,cutoffPrice,priceDecreaseStep,suspensions,Вид разрешённого использования земельного участка,Границы лесного участка,Площадь земельного участка,Площадь земельного участка в соответствии,Регистрационный номер ЕГРОКН,Условный номер земельного участка,Назначение земельного участка,Кадастровый номер земельного участка,Обременения прав на земельный участок,Максимально допустимые параметры разрешенного строительства,Вид договора,Адрес и время приема граждан,Информация об обременениях лесных участков,Информация об ограничении использования лесов,Для сельскохозяйственных организаций,Ежегодный платеж за объект,Платеж за право заключить договор,Собственник имущества,Реквизиты решения о проведении торгов,Вид предмета торгов,Права на земельный участок,Ежемесячный платеж за объект,"Право лица, с которым заключен",Порядок и сроки уплаты покупной,Возможна субаренда,Информация об объеме подлежащей заготовке,Информация о состоянии земельного участка,"Условия договора, заключаемого по результатам",Реквизиты решения об утверждении проекта,Цель предоставления земельного участка,Минимальный объем осуществления жилищного строительства,Льгота по арендной плате,Информация о сроках и порядке,Дата принятия решения о проведении,Требования к параметрам и характеристикам,Порядок возврата задатка,Для граждан,Возможность подключения (технологического присоединения) объектов,Основание реализации имущества,"Обременения, ограничения",Сведения о собственнике (правообладателе) земельного,Порядок оплаты,"Порядок ознакомления с имуществом, иной",Порядок ознакомления с имуществом,Срок и порядок внесения задатка,Основание реализации имущества,Для субъектов малого и среднего,Сведения о решении о проведении,Срок действия договора - месяцев,Обременения реализуемого имущества,Только для крестьянских (фермерских) хозяйств,Срок заключения договора,Только для МСП,"Информация об официальном сайте, на",Предмет торга,Срок аренды,Начальная цена за кв.м,Для крестьянских (фермерских) хозяйств,Минимально допустимые параметры разрешенного строительства,Информация об обязательствах по приведению,Информация о порядке внесения участниками,Обременения продаваемого имущества,Срок действия договора - дней,Адрес сайта в информационно-телекоммуникационной сети,Целевое назначение государственного или муниципального,Сдача имущества в субаренду,Размер взимаемой с победителя аукциона,Информация о выявленных в отношении,Ограничения в использовании земельного участка,Срок действия договора - лет,Участниками аукциона могут являться только,Информация о сроке аренды лесного,Информация об обязательствах по сносу,Перечисление задатка,Особенности объекта торгов,Только для граждан,Ограничения прав на земельный участок
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21000000160000000069_1,Прием заявлений о намерении участвовать в аукц...,SUBMISSION,Сообщение о предоставлении (реализации),21000000160000000069,1,Аренда и продажа земельных участков,52,Прием заявлений о намерении участвовать в аукц...,['64edf46105f9fa758111de88'],"[{'fileId': '64edf46105f9fa758111de88', 'signa...","[{'fileId': '64edf45355ca906d57ab7378', 'signa...",643,Земли сельскохозяйственного назначения,МСК,180,Иная собственность,"обл Ниже

In [478]:
torgi_df.to_csv(torgi_output_filename, index_label="id")

In [485]:
# torgi_df = pd.read_csv(torgi_output_filename, index_col='id')
# torgi_df.shape

D:\_Work\_Projects\_Conda\DLS2\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (51,54,63,77,80,85,87,89,95,116,122,123,125,128) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(9996, 131)